* [#1](https://github.com/salgo60/Naturreservat-Sverige/issues/1) 
* denna Notebook [syncWDOSM](https://github.com/salgo60/Naturreservat-Sverige/blob/main/Notebook/syncWDOSM.ipynb)

Lesson learned massa timeout och missar en hel del...
* a) skapar nya cache bibliotek så dom kan köras om

### Update
qid --> OSM Wikidata 
?NVRID --> OSM ref:NVRID

### TODO 

1) objekt med 2 OSM way pga att OSM har skapat 2 OSM way objekt som borde vara ett OSM rel...
2) det finns WD objekt som upphört --> dom skall inte finnas på OSM

Kanske borde ha statistik 
* klassificerar varje batch-resultat i:
  * SAFE_TO_AUTO_WRITE
  * NEEDS_OSM_STRUCTURAL_FIX
  * INVALID_WD_OBJECT
  * OVERPASS_ERROR
  * NO_MATCH

In [1]:

import time
from datetime import datetime 
import glob
import pandas as pd
from datetime import date

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))



Start: 2026-02-13 21:47:30


In [2]:
import requests
import json
import time
import random
import os
from SPARQLWrapper import SPARQLWrapper, JSON

# -------------------------
# CONFIG (snäll!)
# -------------------------
BATCH_SIZE = 10
BASE_SLEEP = 30  # sek mellan batchar

OVERPASS_SERVERS = [
    "https://overpass-api.de/api/interpreter",
    "https://lz4.overpass-api.de/api/interpreter",
    "https://z.overpass-api.de/api/interpreter"
]

#CACHE_DIR = "overpass_cache2"
CACHE_DIR = f"overpass_cache_{time.strftime('%Y%m%d')}"
os.makedirs(CACHE_DIR, exist_ok=True)

# -------------------------
# 1. HÄMTA WIKIDATA
# -------------------------
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)

query = """
SELECT ?item ?NVRID ?r ?w  WHERE {
  ?item wdt:P31 wd:Q179049;
        wdt:P17 wd:Q34.
  OPTIONAL {?item wdt:P402 ?r}
  OPTIONAL {?item wdt:P3613 ?NVRID}
  OPTIONAL {?item wdt:P10689 ?w}
  FILTER(!BOUND(?r) && !BOUND(?w))
}
"""

sparql.setQuery(query)
res = sparql.query().convert()

items = []

for b in res["results"]["bindings"]:
    qid = b["item"]["value"].split("/")[-1]

    nvrid = None
    if "NVRID" in b:
        nvrid = b["NVRID"]["value"]

    items.append({
        "qid": qid,
        "nvrid": nvrid
    })

print("Totala objekt:", len(items))

# -------------------------
# Helpers
# -------------------------
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n], i//n


def overpass_query(batch):
    wikidata_ids = [i["qid"] for i in batch if i["qid"]]
    nvrids = [i["nvrid"] for i in batch if i["nvrid"]]
    if not wikidata_ids and not nvrids:
        return None

    parts = []

    for q in wikidata_ids:
        parts.extend([
            f'node["wikidata"="{q}"];',
            f'way["wikidata"="{q}"];',
            f'relation["wikidata"="{q}"];'
        ])

    for n in nvrids:
        parts.extend([
            f'node["ref:NVRID"="{n}"];',
            f'way["ref:NVRID"="{n}"];',
            f'relation["ref:NVRID"="{n}"];'
        ])

    return f"""
    [out:json][timeout:120];
    (
        {' '.join(parts)}
    );
    out tags center;
    """ 
    
def polite_request(query, retries=5):
    for attempt in range(retries):

        server = random.choice(OVERPASS_SERVERS)

        try:
            r = requests.post(
                server,
                data=query,
                timeout=180,
                headers={"User-Agent": "Wikidata-NVRID-Resolver/1.0 salgo60@msn.com"}
            )

            if r.status_code == 200:
                return r.json()

            if r.status_code in (429, 504):
                wait = (2 ** attempt) * 30
                print(f"Server stress → väntar {wait}s")
                time.sleep(wait)
                continue

        except Exception as e:
            wait = (2 ** attempt) * 30
            print(f"Fel {e} → väntar {wait}s")
            time.sleep(wait)

    return None

# -------------------------
# RESUME LOGIC
# -------------------------
existing = []
for f in os.listdir(CACHE_DIR):
    if f.startswith("batch_") and f.endswith(".json"):
        try:
            n = int(f.split("_")[1].split(".")[0])
            existing.append(n)
        except:
            pass

start_batch = max(existing) + 1 if existing else 0
print("Resumerar från batch:", start_batch)


# -------------------------
# 2. Kör batchar
# -------------------------
for batch, idx in chunks(items, BATCH_SIZE):

    if idx < start_batch:
        continue

    cache_file = f"{CACHE_DIR}/batch_{idx}.json"

    if os.path.exists(cache_file):
        print(f"Batch {idx} redan klar, hoppar över")
        continue

    print(f"\nBatch {idx} ({len(batch)} QID)")

    q = overpass_query(batch) 
    if not q:
        continue
    result = polite_request(q)

    if result:
        with open(cache_file, "w") as f:
            json.dump(result, f)

        hits = len(result.get("elements", []))
        print(f"Träffar: {hits}")

    sleep_time = BASE_SLEEP + random.randint(0,15)
    print(f"Sover {sleep_time}s...")
    time.sleep(sleep_time)

print("\nKLART 🎉")


Totala objekt: 1456
Resumerar från batch: 0

Batch 0 (10 QID)
Träffar: 2
Sover 40s...

Batch 1 (10 QID)
Träffar: 0
Sover 39s...

Batch 2 (10 QID)
Träffar: 0
Sover 44s...

Batch 3 (10 QID)
Server stress → väntar 30s
Server stress → väntar 60s
Träffar: 0
Sover 33s...

Batch 4 (10 QID)
Träffar: 0
Sover 36s...

Batch 5 (10 QID)
Server stress → väntar 30s
Träffar: 1
Sover 35s...

Batch 6 (10 QID)
Träffar: 0
Sover 43s...

Batch 7 (10 QID)
Träffar: 4
Sover 38s...

Batch 8 (10 QID)
Träffar: 6
Sover 33s...

Batch 9 (10 QID)
Träffar: 4
Sover 38s...

Batch 10 (10 QID)
Träffar: 1
Sover 31s...

Batch 11 (10 QID)
Träffar: 0
Sover 35s...

Batch 12 (10 QID)
Träffar: 0
Sover 41s...

Batch 13 (10 QID)
Server stress → väntar 30s
Träffar: 0
Sover 32s...

Batch 14 (10 QID)
Server stress → väntar 30s
Träffar: 1
Sover 30s...

Batch 15 (10 QID)
Server stress → väntar 30s
Server stress → väntar 60s
Träffar: 2
Sover 45s...

Batch 16 (10 QID)
Träffar: 1
Sover 43s...

Batch 17 (10 QID)
Server stress → väntar 30s


In [9]:
def wd_has_property(qid, prop):
    query = f"""
    ASK {{
      wd:{qid} wdt:{prop} ?value .
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    try:
        result = sparql.query().convert()
        #print(query,result)
        return result["boolean"]
    except Exception:
        return True  # fail safe: assume exists


In [10]:
qs_lines = []
CACHE_DIR = "overpass_cache"

for f in os.listdir(CACHE_DIR):
    if not f.endswith(".json"):
        continue

    with open(os.path.join(CACHE_DIR, f)) as fh:
        data = json.load(fh)

    for el in data.get("elements", []):
        tags = el.get("tags", {})
        qid = tags.get("wikidata")
        #print(el)
        if not qid:
            continue

        if el["type"] == "relation":
            if not wd_has_property(qid, "P402"):
                qs_lines.append(f'{qid}\tP402\t"{el["id"]}"')

        elif el["type"] == "way":
            if not wd_has_property(qid, "P10689"):
                qs_lines.append(f'{qid}\tP10689\t"{el["id"]}"')

# spara
quickstatements_file = f"quickstatements_{time.strftime('%Y%m%d')}.txt"
with open(quickstatements_file, "w") as f:
    f.write("\n".join(qs_lines))

print(f"KLART – {quickstatements_file} skapad")



    ASK {
      wd:Q124616427 wdt:P10689 ?value .
    }
     {'head': {}, 'boolean': True}

    ASK {
      wd:Q124624122 wdt:P10689 ?value .
    }
     {'head': {}, 'boolean': True}

    ASK {
      wd:Q124619257 wdt:P10689 ?value .
    }
     {'head': {}, 'boolean': True}

    ASK {
      wd:Q124619221 wdt:P10689 ?value .
    }
     {'head': {}, 'boolean': True}

    ASK {
      wd:Q124619944 wdt:P10689 ?value .
    }
     {'head': {}, 'boolean': True}

    ASK {
      wd:Q124620764 wdt:P10689 ?value .
    }
     {'head': {}, 'boolean': True}

    ASK {
      wd:Q55619671 wdt:P10689 ?value .
    }
     {'head': {}, 'boolean': True}

    ASK {
      wd:Q55619300 wdt:P10689 ?value .
    }
     {'head': {}, 'boolean': True}

    ASK {
      wd:Q55692884 wdt:P10689 ?value .
    }
     {'head': {}, 'boolean': True}

    ASK {
      wd:Q55633147 wdt:P10689 ?value .
    }
     {'head': {}, 'boolean': True}

    ASK {
      wd:Q55623677 wdt:P10689 ?value .
    }
     {'head': {}, 'boolean':

In [11]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

Date: 2026-02-14 08:00:01
Total time elapsed: 36750.78 seconds
